# Json 학습

**시스템에 들어와서 → 규격화되고 → 대규모로 확장되는** 논리적 순서에 따른 학습

| **단계**  | **주제**                    | **학습 목표**                                                     |
| ------- | ------------------------- | ------------------------------------------------------------- |
| **1단계** | **평탄화 (Flattening)**    | 복잡하게 얽힌 JSON에서 필요한 데이터를 추출하고 표 형식으로 만드는 법을 익힙니다.              |
| **2단계** | **규격화 (Normalization)**  | Pydantic을 사용해 데이터의 타입을 검증하고, 서로 다른 명명 규칙(camelCase 등)을 통일합니다. |
| **3단계** | **효율화 (Efficiency)**     | 대용량 데이터를 처리할 때 메모리 부하를 줄이는 파싱 기법을 배웁니다.                       |

**단계별 의미**
1. 데이터 정형화 단계(데이터 형태)
2. 데이터 규격화 단계(데이터 자료형)
3. 데이터 효율화 단계(데이터 효율 처리)

## 1단계: 평탄화(Flattening)

평탄화(Flattening)란?
데이터 분석 도구(Pandas 등)나 데이터베이스는 보통 표(Table) 형태를 선호합니다.

하지만 위 JSON처럼 info 안에 contact가 있고, 그 안에 email이 있는 구조는 표의 한 칸에 넣기가 매우 까다롭습니다.

그래서 우리는 이 계층을 깨고 **'한 줄의 데이터'** 로 펼치는 작업이 필요합니다.

보통 계층을 구분하기 위해 점(.)을 사용하곤 하죠.

In [1]:
# 아래의 중첩된 구조(Nested Structure)의 Json 형태
# email 컬럼의 계층 구조 info.contact.email
{
  "id": "USR_001",
  "info": {
    "name": "김철수",
    "contact": {
      "email": "chulsoo@example.com",
      "phone": "010-1234-5678"
    }
  },
  "tags": ["developer", "python"]
}

{'id': 'USR_001',
 'info': {'name': '김철수',
  'contact': {'email': 'chulsoo@example.com', 'phone': '010-1234-5678'}},
 'tags': ['developer', 'python']}

In [7]:
import pandas as pd

data = {
  "id": "USR_001",
  "info": {
    "name": "김철수",
    "contact": {
      "email": "chulsoo@example.com",
      "phone": "010-1234-5678"
    }
  },
  "tags": ["developer", "python"]
}

df = pd.json_normalize(data)
print(f"Flattening(평탄화): \n{df.columns}")

Flattening(평탄화): 
Index(['id', 'tags', 'info.name', 'info.contact.email', 'info.contact.phone'], dtype='object')


## 2단계: 규격화 (Normalization)

데이터를 DataFrame(표) 형태로 만들었습니다.

이 데이터들의 형식이 올바른지 검사하고 이름을 파이썬 스타일로 바꿀 차례입니다.

Pydantic 모델을 만들 때 Field의 alias 설정을 사용하면,

"데이터를 읽어올 때는 createdAt을 찾고, 파이썬 변수로는 created_at을 쓰겠다"라고 선언할 수 있습니다.

In [8]:
# Pydantic Alias

from pydantic import BaseModel, Field

class User(BaseModel):
    user_id: str = Field(alias="id")           # 'id'를 'user_id'로 매핑
    user_name: str = Field(alias="name")       # 'name'을 'user_name'로 매핑
    created_at: str = Field(alias="createdAt") # 'createdAt'을 'created_at'으로 매핑

In [9]:
from pydantic import BaseModel, Field, PositiveInt

class UserProfile(BaseModel):
    # Field의 alias를 사용하여 camelCase를 snake_case로 매핑
    # PositiveInt(양의 정수)를 사용하여 0보다 큰 정수임을 보장
    user_age: PositiveInt = Field(alias="userAge")

## 3단계: 효율화(Efficiency)

대용량 JSON 처리 방식: 몇 천만 줄의 로그 중 특정 필드 `user_id`, `action`만 필요한 경우

메모리를 아끼기 위하여 데이터를 한 줄씩 읽으면서 필요한 필드만 즉시 추출하고자 하는 경우

**Streaming & Filtering 기법**
보통 큰 파일은 한 번에 다 읽지 않고 `for line in file:` 형식을 사용합니다.

이 때 각 줄(Line)에서 필요한 것만 골라내면 메모리 점유율을 획기적으로 낮출 수 있습니다.

특히 필요한 데이터만 필터링 시 보안 관리에도 유용합니다.

In [16]:
import json

# 대용량 로그 파일이 있다고 가정(log.jsonl - Json Lines 형식)
# 각 줄이 하나의 Json 객체인 경우

"""
# 객체 예시
{
  "user_id": "U12345",
  "action": "login",
  "ip_address": "192.168.0.1",
  "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)...",
  "session_id": "sess_987654321",
  "server_internal_code": "ERR_NONE_001",
  "timestamp": "2026-01-06T15:00:00Z"
}

# 객체 파싱
with open("large_logs.json", "r", encoding="utf-8") as file:
    for line in file:
        # 1. 한 줄씩 읽어서 파싱(Streaming)
        record = json.loads(line)

        # 2. 필요한 필드만 추출(Filtering)
        user_id = record.get("user_id")  # 결과: "U12345"
        action = record.get("action")    # 결과: "login"

        # 3. 추출한 데이터만 처리하고 나머지는 record 변수와 함께 소멸!
        process_data(user_id, action)
"""        

SyntaxError: invalid syntax (1735371117.py, line 30)

## 요약 및 완성형 파이썬 스크립트

| **단계**  | **기법**                  | **목적**             | **핵심 도구**               |
| ------- | ----------------------- | ------------------ | ----------------------- |
| **1단계** | **평탄화 (Flattening)**    | 중첩 구조를 표 형태로 변환    | `pd.json_normalize`     |
| **2단계** | **규격화 (Normalization)** | 이름 통일 및 데이터 유효성 검증 | `Pydantic (Alias)`      |
| **3단계** | **효율화 (Efficiency)**    | 메모리 절약 및 보안 강화     | `Streaming & Filtering` |

**완성형 파이썬 스크립트**
1. 데이터 수집: 대용량 로그 파일(`jsonl` 형식)을 한 줄씩 읽습니다.
2. 데이터 매핑: Pydantic의 `alias`를 사용하여 `camelCase`를 `snake_case`로 바꿉ㄴ디ㅏ.
3. 데이터 검증: 필수 필드가 있는지, 값의 타입이 맞는지 검사합니다.
4. 필터링 및 추출: 필요한 정보만 골라내어 메모리를 아낍니다.

**Pydantic V2 `model_config`**
| **옵션명**                    | **설명**                                               | **비고**                            |
| -------------------------- | ---------------------------------------------------- | --------------------------------- |
| **`populate_by_name`**     | 필드 이름(snake_case)과 별칭(alias) 모두로 데이터를 읽을 수 있게 허용합니다. | API 통신 시 필수적                    |
| **`str_strip_whitespace`** | 문자열 데이터 앞뒤의 공백을 자동으로 제거합니다.                          | 데이터 정제에 유용                       |
| **`extra`**                | 모델에 정의되지 않은 추가 필드가 들어왔을 때의 처리를 결정합니다.                | `'ignore'`, `'allow'`, `'forbid'` |
| **`frozen`**               | 모델을 생성한 후 값을 변경할 수 없게 만듭니다 (Immutable).              | 보안 및 안정성 강화                     |

**Config Option**
- **`ignore` (기본값):** 정의되지 않은 값은 그냥 무시하고 버립니다.
- **`allow`:** 정의되지 않은 값도 일단 다 받아둡니다. (데이터 유실을 막고 싶을 때)
- **`forbid`:** 정의되지 않은 값이 하나라도 들어오면 에러를 냅니다. (엄격한 규격 관리가 필요할 때)

In [25]:
import json
from pydantic import BaseModel, Field, ValidationError, ConfigDict

# 1. 규격화: 데이터 모델 정의 (Pydantic)
class UserLog(BaseModel):
    # alias를 사용하여 API 명칭과 파이썬 변수명을 매핑합니다.
    model_config = ConfigDict(populate_by_name=True)
    user_id: str = Field(alias="userId")
    action_type: str = Field(alias="actionType")

# 2. 효율화: 스트리밍 및 필터링 함수
def process_json_logs(json_lines):
    processed_data = []
    
    for line in json_lines:
        try:
            # 한 줄씩 파싱 (Streaming)
            raw_data = json.loads(line)
            
            # Pydantic 모델로 변환 (Validation & Normalization)
            # 이 과정에서 camelCase인 userId가 snake_case인 user_id로 변합니다.
            log = UserLog(**raw_data)
            
            # 3. 파싱 필요한 필드만 리스트에 담기 (Filtering)
            processed_data.append({
                "id": log.user_id,
                "action": log.action_type
            })
            
        except (ValidationError, json.JSONDecodeError) as e:
            # 비정상 데이터는 건너뛰어 시스템 중단을 방지합니다.
            print(f"Skipping invalid log: {e}")
            
    return processed_data

# --- 실행 예시 ---
raw_logs = [
    '{"userId": "U001", "actionType": "login", "ip": "1.1.1.1"}',
    '{"userId": "U002", "actionType": "logout", "timestamp": 1234567}'
]

results = process_json_logs(raw_logs)
print(results)

[{'id': 'U001', 'action': 'login'}, {'id': 'U002', 'action': 'logout'}]
